Esse código requer scikit-learn < 0.24

In [1]:
!pip install scikit-learn==0.23.2

# Criação de funções de obtenção de labels

In [2]:
import pandas as pd

df = pd.read_csv('dodf_atos_pessoal_v1.csv', encoding = 'latin-1')
df

,id_ato,id_dodf,num_doc_dodf,data_doc_dodf,tipo_rel,id_rel,anotador_rel,texto_rel,tipo_ent,id_ent,anotador_ent,texto_ent
0,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,simbolo,4678,DODFMiner,DFA-14
1,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,orgao,4676,DODFMiner,Governadoria do Distrito Federal
2,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,nome,4677,DODFMiner,MARIA GERLANIA SOARES
3,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,hierarquia_lotacao,4650,DODFMiner,"Assessoria, do Gabinete, da Administracao Regi..."
4,34_224.24.10.2014-R5,34_224.24.10.2014,224,24.10.2014,Ato_Nomeacao_Comissionado,R5,lygia_paloma,NOMEAR MARIA GERLANIA SOARES para exercer o Ca...,cargo_comissionado,7956,lygia_paloma,Assessor
...,...,...,...,...,...,...,...,...,...,...,...,...
36035,83_216.30.10.8-R231,83_216.30.10.8,216,30.10.8,Ato_Substituicao,R231,marco_antonio,"DESIGNAR DURVALINA SILVA RABELO, matricula 26....",simbolo_objeto_substituicao,7107,marco_antonio,DFG-04
36036,83_216.30.10.8-R231,83_216.30.10.8,216,30.10.8,Ato_Substituicao,R231,marco_antonio,"DESIGNAR DURVALINA SILVA RABELO, matricula 26....",orgao,7110,marco_antonio,Procuradoria-Geral do Distrito Federal
36037,83_216.30.10.8-R231,83_216.30.10.8,216,30.10.8,Ato_Substituicao,R231,marco_antonio,"DESIGNAR DURVALINA SILVA RABELO, matricula 26....",data_inicial,7111,marco_antonio,16/10
36038,83_216.30.10.8-R231,83_216.30.10.8,216,30.10.8,Ato_Substituicao,R231,marco_antonio,"DESIGNAR DURVALINA SILVA RABELO, matricula 26....",data_final,7112,marco_antonio,23/\n10/2008


In [3]:
try:
    from dodflib.core.preprocess import IOBifyer
except ModuleNotFoundError:
    from dodflib.core.preprocess import IOBifyer
from dodflib.core.feature_extraction import Tokenizer

In [4]:
def filtra_ato(df, ato):
    df_ato = df.loc[df['tipo_rel'] == ato]
    df_ato.reset_index(drop = True)
    df_ato = df_ato.filter(['id_ato', 'tipo_ent', 'texto_ent'])
    df_ato = df_ato.reset_index(drop = True)
    act_types = df['tipo_rel'].value_counts()
    return df_ato, act_types
    
def obter_entidades(df, act_types):
    
    row_list = []

    id_atual = df['id_ato'][0]
    d = {}

    for row in df.iterrows():
        id_ato = row[1]['id_ato']
        tipo_ent = row[1]['tipo_ent']
        texto_ent = row[1]['texto_ent']

        if tipo_ent in act_types:
            tipo_ent = 'text'

        if id_ato != id_atual:
            row_list.append(d)
            d = {}
            id_atual = id_ato

        d[tipo_ent] = texto_ent
    
    return pd.DataFrame(row_list)

def obter_IOB(df):
    
    text_notnull = df['text'].dropna()
    df = df.loc[text_notnull.index]
    df = df.reset_index(drop = True)
    
    tokenizer = Tokenizer()
    labels_list = []

    text_index = 0
    for i in df.columns:
        if i == 'text':
            break
        text_index += 1
    
    for index, row in df.iterrows():
        labels = IOBifyer.generate_IOB_labels(row, text_index, tokenizer)
        labels_list.append(labels)
        assert len(labels) == len(tokenizer(row['text'])), 'index %d: %d != %d' %(index, len(labels), len(tokenizer(row['text'])))

    df['labels'] = labels_list
    df['labels'] = df['labels'].apply(lambda x: ' '.join(x))
    return df

In [5]:
df_ato, act_types = filtra_ato(df, 'Ato_Substituicao')
df_ato = obter_entidades(df_ato, act_types)
df_com_labels = obter_IOB(df_ato)

In [6]:
df_com_labels

,nome_substituto,matricula_substituto,cargo_substituto,simbolo_substituto,hierarquia_lotacao,orgao,data_inicial,data_final,motivo,nome_substituido,matricula_substituido,cargo_objeto_susbtituicao,simbolo_objeto_substituicao,text,matricula_SIAPE,labels
0,ALCIDES GOMES DE ARAUJO FILHO,1.655.773-5,"Gerente,",DFG-14,"Diretoria de Administracao Geral,","Administracao Regional do Lago \nSul, da Casa ...",1o,10 de setembro de 2014,ferias e sem \nprejuizo de suas atribuicoes,"KARLA MENDES BATISTA,",1.660.492-X,Diretora,CNE-07,"DESIGNAR ALCIDES GOMES DE ARAUJO FILHO, matric...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
1,EDUARDO DE MIRANDA DUARTE,16533844,Encarregado do Nucleo de Material e Patrimonio,DFG-05,NaN,"Administracao Regional do Riacho Fundo, \nda C...",10/11/2014,09/12/2014,ferias de regulamentar do titular.,"LUIZ CARLOS DE SA,",16607406,Diretor de Administracao Geral,CNE-07,"DESIGNAR EDUARDO DE MIRANDA DUARTE, matricula ...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
2,CARMEM LUCIA DE CARVALHO SANTOS,40.579-5,Tecnico Juridico,NaN,Diretor da Diretoria de Arrecadacao \nTributar...,Procuradoria Geral do Distrito Federal,16/10/2014,"25/10/2014,",licenca medica do titular,MARCELO DE OLIVEIRA PEREIRA,34.437-0,"Natureza Especial,",CNE-07,"DESIGNAR CARMEM LUCIA DE CARVALHO SANTOS, matr...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
3,ROSINEIDE RODRIGUES MUNIZ,41.616-9,Agente \nJuridico,NaN,Assessor da Assessoria Executiva da Coordenaca...,"Procuradoria Geral do Distrito Federal,",28/10/2014,06/11/2014,ferias regulamentares da titular.,KELLY DE SOUZA MORAIS,168.642-9,Comissao,DFA-14,"DESIGNAR ROSINEIDE RODRIGUES MUNIZ, matricula ...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
4,DARIO BALBINO DA SILVA,91.994-2,NaN,NaN,"Gerencia de Suprimentos e Servicos Gerais, da ...",Secretaria de Estado de Obras,06/10/2008,04/11/\n2008,ferias do titular,SEBASTIAO\nABDALA,162.088-6,Chefe,DFG-09,"DESIGNAR DARIO BALBINO DA SILVA, matricula 91....",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1242,HELDER PAIS DE OLIVEIRA,137.542-3,CE Enfermeiro,NaN,"Gerencia de Enferma-\ngem, da Diretoria Geral ...",NaN,29.12 .2008,17.01.2009,ferias do Titular,VENERY RODRIGUES GALVAO,NaN,Supervisor de Enfermagem,DFG-05,"DESIGNAR HELDER PAIS DE OLIVEIRA, CE Enfermei...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
1243,ZANA MARIA SILVELLO,134.496-X,Nutricionista,NaN,Nu-\ncleo de Nutricao e Dietetica,Hospital Regional do Paranoa,08/12/2008,27/12/\n2008,virtude de ferias,MARJORIE LIMA BRITO FERREIRA,154.225-7,chefia,NaN,"DESIGNAR ZANA MARIA SILVELLO, matricula 134.49...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
1244,AMILTON LOIOLA MENEZES,0000233-X,NaN,NaN,Gerencia de Pessoal/CAO/ FE-\nPECS,NaN,12.11.08,26.11.08,ferias da titular,CELIA\nMILHOMEM AMARAL,0080927-6,Gerente,DFG-11,"DESIGNAR AMILTON LOIOLA MENEZES, matricula 000...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...
1245,EDSON MOULIN PEDROSA,0084933-2,NaN,NaN,Gerencia de Atividades Gerais/\nCAO/FEPECS,NaN,24.11.08,13.12.08,ferias do\ntitular,RUBENS\nBATISTA DE OLIVEIRA,0084373-3,Gerente,DFG-11,"DESIGNAR EDSON MOULIN PEDROSA, matricula 00849...",NaN,O B-nome_substituto I-nome_substituto I-nome_s...


In [7]:
def obter_labels(df, ato):
    df_ato, act_types = filtra_ato(df, ato)
    df_ato = obter_entidades(df_ato, act_types)
    df_com_labels = obter_IOB(df_ato)
    return df_com_labels

# Funções de Classificação

In [8]:
import pandas as pd
import sklearn_crfsuite

In [9]:
def tokenizar_labels(df):
    tokenizer = Tokenizer()
    data = df
    x = []
    y = []
    for row in range(len(data)):
        if pd.notna(data['text'][row]) and pd.notna(data['labels'][row]):
            x.append(tokenizer(data['text'][row]))
            y.append(data['labels'][row].split())
    return x, y

def get_features_(sentence):
        """Create features for each word in act.
        Create a list of dict of words features to be used in the predictor module.
        Args:
            act (list): List of words in an act.
        Returns:
            A list with a dictionary of features for each of the words.
        """
        sent_features = []
        for i in range(len(sentence)):
            word_feat = {
                'word': sentence[i].lower(),
                'capital_letter': sentence[i][0].isupper(),
                'all_capital': sentence[i].isupper(),
                'isdigit': sentence[i].isdigit(),
                'word_before': sentence[i].lower() if i == 0 else sentence[i-1].lower(),
                'word_after:': sentence[i].lower() if i+1 >= len(sentence) else sentence[i+1].lower(),
                'BOS': i == 0,
                'EOS': i == len(sentence)-1
            }
            sent_features.append(word_feat)
        return sent_features
    
def get_features(x):
    for i in range(len(x)):
        x[i] = get_features_(x[i])

In [10]:
def obter_xy(df):
    x, y = tokenizar_labels(df)
    get_features(x)
    return x, y

# Randomized Search

In [11]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import numpy as np

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.model_selection import KFold

In [12]:
def RandomizedSearchCRF(X, y, labels):
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        max_iterations=100,
        all_possible_transitions=True
    )
    
    params_space = {
        'c1': scipy.stats.expon(scale=0.5),
        'c2': scipy.stats.expon(scale=0.05),
        'epsilon': scipy.stats.expon(scale=1e-5),
    }
    
    f1_scorer = make_scorer(metrics.flat_f1_score,
                            average='weighted', labels=labels)

    clf = RandomizedSearchCV(crf, params_space, cv = 5, n_jobs = -1, n_iter = 10, scoring = f1_scorer, verbose = 10)
    clf.fit(X, y)
    
    return clf.best_estimator_

In [13]:
def KFoldCrossValidation(X, y, n_splits = 5):
    kf = KFold(n_splits = n_splits, random_state = 87, shuffle = True)
    
    results = []
    
    labels = list(np.unique(np.concatenate([np.array(i) for i in y])))
    labels.remove('O')
    
    for train, test in kf.split(X, y):
        X_train, y_train = X[train], y[train]
        X_test, y_test = X[test], y[test]
        
        clf = RandomizedSearchCRF(X_train, y_train, labels)
        
        y_true, y_pred = y_test, clf.predict(X_test)
        score = metrics.flat_f1_score(y_true, y_pred, average='weighted', labels=labels)
        report = metrics.flat_classification_report(y_true, y_pred, labels = labels, digits = 3)
        
        results.append({'clf': clf, 'score': score, 'report': report})
        
    results.sort(key = lambda x: x['score'])
    
    middle = ( n_splits + 1 ) // 2
    
    return results[middle]

# Treinamento de todos os modelos

In [14]:
tipos_de_ato = df['tipo_rel'].value_counts().keys()

scores = {}

In [15]:
for ato in tipos_de_ato:
    size = len(obter_labels(df, ato))
    print('%s: %d' %(ato, size))

Ato_Substituicao: 1247
Ato_Nomeacao_Comissionado: 784
Ato_Exoneracao_Comissionado: 667
Ato_Retificacao_Efetivo: 436
Ato_Retificacao_Comissionado: 91
Ato_Tornado_Sem_Efeito_Exo_Nom: 89
Ato_Cessao: 75
Ato_Abono_Permanencia: 59
Ato_Nomeacao_Efetivo: 23
Ato_Exoneracao_Efetivo: 29
Ato_Reversao: 29
Ato_Tornado_Sem_Efeito_Apo: 9


In [16]:
%%time
for ato in tipos_de_ato:
    print('Iniciando %s' %ato)
    X, y = obter_xy(obter_labels(df, ato))
    r = KFoldCrossValidation(np.array(X), np.array(y))
    scores[ato] = r

Iniciando Ato_Substituicao
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/home/felipe/.local/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  6.9min finished
/home/felipe/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=['B-cargo_objeto_susbtituicao', 'B-cargo_substituto', 'B-data_final', 'B-d

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  6.9min finished


Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.0s


[CV]  c1=0.33809118378814024, c2=0.06568001128738186, epsilon=2.415440826993153e-06, score=0.785, total=   0.2s
[CV] c1=0.33809118378814024, c2=0.06568001128738186, epsilon=2.415440826993153e-06 
[CV]  c1=0.33809118378814024, c2=0.06568001128738186, epsilon=2.415440826993153e-06, score=0.800, total=   0.2s
[CV] c1=0.028755279955154006, c2=0.009526230975031368, epsilon=9.611944032311512e-06 
[CV]  c1=0.028755279955154006, c2=0.009526230975031368, epsilon=9.611944032311512e-06, score=0.924, total=   0.2s
[CV] c1=1.2973495472783296, c2=0.053403588240693134, epsilon=5.7756243586632724e-08 
[CV]  c1=1.2973495472783296, c2=0.053403588240693134, epsilon=5.7756243586632724e-08, score=0.621, total=   0.2s
[CV] c1=0.7255217618198051, c2=0.1104898272976364, epsilon=2.5859849233351865e-06 
[CV]  c1=0.7255217618198051, c2=0.1104898272976364, epsilon=2.5859849233351865e-06, score=0.754, total=   0.2s
[CV] c1=0.5889355431790967, c2=0.01217278164305062, epsilon=2.4876192709487037e-05 
[CV]  c1=0.58893

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.1s


CPU times: user 8min 19s, sys: 1.8 s, total: 8min 21s
Wall time: 1h 59min 26s


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    6.0s finished


In [17]:
for ato in scores:
    print("%s: %f" %(ato, scores[ato]['score']))

Ato_Substituicao: 0.899528
Ato_Nomeacao_Comissionado: 0.977717
Ato_Exoneracao_Comissionado: 0.967784
Ato_Retificacao_Efetivo: 0.877926
Ato_Retificacao_Comissionado: 0.742843
Ato_Tornado_Sem_Efeito_Exo_Nom: 0.961251
Ato_Cessao: 0.863726
Ato_Abono_Permanencia: 0.839229
Ato_Nomeacao_Efetivo: 0.555332
Ato_Exoneracao_Efetivo: 0.958089
Ato_Reversao: 0.876762
Ato_Tornado_Sem_Efeito_Apo: 0.914706


In [18]:
for ato in scores:
    print("%s: " %ato)
    print(scores[ato]['report'])

Ato_Substituicao: 
                               precision    recall  f1-score   support

  B-cargo_objeto_susbtituicao      0.890     0.906     0.898       233
           B-cargo_substituto      0.854     0.850     0.852       234
                 B-data_final      0.984     0.992     0.988       241
               B-data_inicial      0.976     0.980     0.978       248
         B-hierarquia_lotacao      0.826     0.752     0.787       234
            B-matricula_SIAPE      0.933     0.903     0.918        31
      B-matricula_substituido      0.774     0.831     0.801       177
       B-matricula_substituto      0.876     0.837     0.856       270
                     B-motivo      0.992     0.971     0.981       244
           B-nome_substituido      0.888     0.829     0.857       210
            B-nome_substituto      0.862     0.896     0.879       251
                      B-orgao      0.891     0.925     0.908       186
B-simbolo_objeto_substituicao      0.934     0.996     0.

In [19]:
for ato in scores:
    print("%s: " %ato, scores[ato]['clf'])

Ato_Substituicao:  CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.7283893920049836,
    c2=0.14316115372891863, epsilon=2.0907854957722005e-06, keep_tempfiles=None,
    max_iterations=100)
Ato_Nomeacao_Comissionado:  CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.02550330584323263,
    c2=0.07282711661781041, epsilon=9.894948690591827e-06, keep_tempfiles=None,
    max_iterations=100)
Ato_Exoneracao_Comissionado:  CRF(algorithm='lbfgs', all_possible_transitions=True, c1=1.1746703543258148,
    c2=0.3087617356221705, epsilon=1.5431261551362188e-05, keep_tempfiles=None,
    max_iterations=100)
Ato_Retificacao_Efetivo:  CRF(algorithm='lbfgs', all_possible_transitions=True, c1=1.7598221072859748,
    c2=0.011131771245111985, epsilon=1.1165877737535072e-05,
    keep_tempfiles=None, max_iterations=100)
Ato_Retificacao_Comissionado:  CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.8299148392158164,
    c2=0.007405567020165273, epsilon=3.109872832314453e-05, k

In [20]:
import pickle
for ato in scores:
    path = 'modelos/' + ato + '.pkl'
    with open(path, 'wb') as file:
        pickle.dump(scores[ato]['clf'], file)